In [1]:
from pathlib import Path
import json
import os
import requests
from time import sleep

# def start_session(email,password):
#     s = requests>Session()
#     s.auth = (email,password)
#     r = s.post('https://api.worldquantbrain.com/authentication')
def start_session(username, password):
    s = requests.Session()
    s.auth = (username, password)

    while True:
        try:
            # Send a POST request to the /authentication API
            response = s.post('https://api.worldquantbrain.com/authentication')
            response.raise_for_status()  # Raises an HTTPError if the status is 4xx, 5xx
            break
        except Exception as e:
            print(f"Connection down, trying to login again: {e}")
            sleep(10)

    print(response)
    return s

In [2]:
username = "wanqidu@link.cuhk.edu.cn"
password = "worldquant159753"

s = start_session(username, password)

<Response [201]>


Simulating an Alpha

In [3]:
simulation_data = {
    'type': 'REGULAR',
    'settings': {
        'instrumentType': 'EQUITY',
        'region': 'CHN',
        'universe': 'TOP3000',
        'delay': 1,
        'decay': 5,
        'neutralization': 'SECTOR',
        'truncation': 0.08,
        'pasteurization': 'ON',
        'unitHandling': 'VERIFY',
        'nanHandling': 'ON',
        'language': 'FASTEXPR',
        'visualization': False,
    },
    'regular': 'high-close'
}
simulation_response = s.post('https://api.worldquantbrain.com/simulations', json=simulation_data)

Alpha 回测状态和结果检索

In [4]:

from time import sleep

simulation_progress_url = simulation_response.headers['Location']
finished = False
while True:
    simulation_progress = s.get(simulation_progress_url)
    if simulation_progress.headers.get("Retry-After", 0) == 0:
        break
    print("Sleeping for " + simulation_progress.headers["Retry-After"] + " seconds")
    sleep(float(simulation_progress.headers["Retry-After"]))
print("Alpha done simulating, getting alpha details")
alpha_id = simulation_progress.json()["alpha"]
alpha = s.get("https://api.worldquantbrain.com/alphas/" + alpha_id)
alpha.json()

Sleeping for 2.5 seconds
Sleeping for 2.5 seconds
Sleeping for 2.5 seconds
Sleeping for 2.5 seconds
Alpha done simulating, getting alpha details


{'id': 'gxEQRQg',
 'type': 'REGULAR',
 'author': 'WD77850',
 'settings': {'instrumentType': 'EQUITY',
  'region': 'CHN',
  'universe': 'TOP3000',
  'delay': 1,
  'decay': 5,
  'neutralization': 'SECTOR',
  'truncation': 0.08,
  'pasteurization': 'ON',
  'unitHandling': 'VERIFY',
  'nanHandling': 'ON',
  'language': 'FASTEXPR',
  'visualization': False},
 'regular': {'code': 'high-close', 'description': None, 'operatorCount': 1},
 'dateCreated': '2024-01-29T22:45:54-05:00',
 'dateSubmitted': None,
 'dateModified': '2024-01-30T22:00:01-05:00',
 'name': None,
 'favorite': False,
 'hidden': False,
 'color': 'RED',
 'category': None,
 'tags': [],
 'grade': None,
 'stage': 'IS',
 'status': 'UNSUBMITTED',
 'is': {'pnl': 2384664,
  'bookSize': 20000000,
  'longCount': 855,
  'shortCount': 1871,
  'turnover': 0.2494,
  'returns': 0.0247,
  'drawdown': 0.6577,
  'margin': 0.000198,
  'fitness': 0.05,
  'sharpe': 0.15,
  'startDate': '2012-01-22',
  'checks': [{'name': 'LOW_SHARPE',
    'result':

Record Sets of Alpha Simulations Alpha Pnl

In [5]:
from time import sleep


finished = False
while True:
    pnl  = s.get("https://api.worldquantbrain.com/alphas/"+ alpha_id+"/recordsets/pnl")
    if pnl.headers.get("Retry-After", 0) == 0:
        break
    print("Sleeping for " + pnl.headers["Retry-After"] + " seconds")
    sleep(float(pnl.headers["Retry-After"]))
print("PnL retrieved")
pnl.json()

Sleeping for 1.0 seconds
PnL retrieved


{'schema': {'name': 'pnl',
  'title': 'PnL',
  'properties': [{'name': 'date', 'title': 'Date', 'type': 'date'},
   {'name': 'pnl', 'title': 'PnL', 'type': 'amount'}]},
 'records': [['2012-01-30', 14520.0],
  ['2012-01-31', 5831.0],
  ['2012-02-01', 27758.0],
  ['2012-02-02', 44082.0],
  ['2012-02-03', 23661.0],
  ['2012-02-06', 29201.0],
  ['2012-02-07', 62846.0],
  ['2012-02-08', 109677.0],
  ['2012-02-09', 106651.0],
  ['2012-02-10', 111026.0],
  ['2012-02-13', 167111.0],
  ['2012-02-14', 140780.0],
  ['2012-02-15', 70805.0],
  ['2012-02-16', 7592.0],
  ['2012-02-17', 41661.0],
  ['2012-02-20', 83715.0],
  ['2012-02-21', 164066.0],
  ['2012-02-22', 233108.0],
  ['2012-02-23', 253428.0],
  ['2012-02-24', 219408.0],
  ['2012-02-27', 249584.0],
  ['2012-02-28', 223027.0],
  ['2012-02-29', 222742.0],
  ['2012-03-01', 225368.0],
  ['2012-03-02', 244541.0],
  ['2012-03-05', 277447.0],
  ['2012-03-06', 272313.0],
  ['2012-03-07', 249908.0],
  ['2012-03-08', 253387.0],
  ['2012-03-09', 3269

In [6]:
pc = pnl.json()

Get Performance Comparsion

In [7]:
import time
def performance_comparison(s, alpha_id, team_id = None, competition = None):
    """
    Returns performance comparison for merged performance check
    """
    if competition is not None:
        part_url = f'competitions/{competition}'
    elif team_id is not None:
        part_url = f'teams/{team_id}'
    else:
        part_url = 'users/self'
    while True:
        result = s.get(
            f"https://api.worldquantbrain.com/{part_url}/alphas/" + alpha_id + "/before-and-after-performance"
        )
        if "retry-after" in result.headers:
            time.sleep(float(result.headers["Retry-After"]))
        else:
            break
    if result.json().get("stats", 0) == 0:
        return {}
    if result.status_code != 200:
        return {}

    return result.json()

In [8]:
new_pc = performance_comparison(s, alpha_id)

In [9]:
new_pc

{'stats': {'before': {'bookSize': 20000000,
   'pnl': 12312064.0,
   'longCount': 1067,
   'shortCount': 979,
   'drawdown': 0.1105,
   'turnover': 0.1807,
   'returns': 0.1294,
   'margin': 0.001432,
   'sharpe': 4.42,
   'fitness': 3.74},
  'after': {'bookSize': 20000000,
   'pnl': 8698288.0,
   'longCount': 1023,
   'shortCount': 1160,
   'drawdown': 0.3181,
   'turnover': 0.188,
   'returns': 0.0834,
   'margin': 0.000887,
   'sharpe': 1.04,
   'fitness': 0.69}},
 'yearlyStats': {'before': {'schema': {'name': 'yearly-stats',
    'title': 'Yearly Stats',
    'properties': [{'name': 'year', 'title': 'Year', 'type': 'year'},
     {'name': 'pnl', 'title': 'PnL', 'type': 'amount'},
     {'name': 'bookSize', 'title': 'Book Size', 'type': 'amount'},
     {'name': 'longCount', 'title': 'Long Count', 'type': 'integer'},
     {'name': 'shortCount', 'title': 'Short Count', 'type': 'integer'},
     {'name': 'turnover', 'title': 'Turnover', 'type': 'percent'},
     {'name': 'sharpe', 'title': '

In [10]:
# new_records = new_pc["pnl"]["records"] #big list
# before = []
# after = []
# for lis in new_records:
#     before.append(lis[1])
#     after.append(lis[2])
new_records = pc["records"] #big list
before = []
after = []
for lis in new_records:
    before.append(lis[1])
    

参考Fitness的逻辑， 自设一个Filtering Scoring

In [22]:
before_after_stats = performance_comparison(s, alpha_id)['stats']

In [24]:
def calculate_score(data):
    returns_drawdown_ratio = min(data['returns']/data['drawdown'],3)

    score = data['sharpe'] = returns_drawdown_ratio / max(data['turnover'],0.125)
    return score

In [53]:
calculate_score(before_after_stats['before'])

6.480579546109022

In [54]:
calculate_score(before_after_stats['after'])

1.3945835312058967

In [25]:
def update_alpha_color(session, alpha_id,color):
    update_data = {"color":color}
    response = session.patch(f"https://api.worldquantbrain.com/alphas/{alpha_id}", json=update_data)
    return response.status_code == 200
if calculate_score(before_after_stats['after'])>calculate_score(before_after_stats['before']):
    update_alpha_color(session = s, alpha_id = alpha_id, color = 'GREEN')
else:
    print("The alpha is not avery good")
    update_alpha_color(session = s, alpha_id = alpha_id, color = 'RED')
    

The alpha is not avery good


参考101 performance 的逻辑， 自设一个Filtering Score

In [31]:
import math
def calculate_score(data,returns,turnover):
    records = data['records']# long list
    drawdown_list = []
    for lis in records:
        drawdown = lis[8]
        drawdown_list.append(drawdown**2)
    new_drawdown = math.sqrt(sum(drawdown_list))
    Burke_ratio = min(returns/new_drawdown,3)

    score = data['sharpe'] = Burke_ratio / max(turnover,0.125)
    return score

In [32]:
def catch_return_turnover(data):
    returns = data['returns']

    turnover = data['turnover']
    return returns, turnover

In [28]:
before_after_yearlyStats = performance_comparison(s, alpha_id)['yearlyStats']

In [29]:
returns1, turnover1 = catch_return_turnover(before_after_stats['before'])
returns2, turnover2 = catch_return_turnover(before_after_stats['after'])


In [33]:
calculate_score(before_after_yearlyStats['before'],returns1,turnover1)

5.8416943767194125

In [9]:
pnl

<Response [200]>

Alpha List Alpha 列表

In [34]:
def get_n_is_alphas(session, total_alphas, limit=100):
    fetched_alphas = []
    offset = 0
    #alphaNum = 0#我自己加的

    while len(fetched_alphas) < total_alphas:
        response = session.get(f"https://api.worldquantbrain.com/users/self/alphas?stage=OS&limit={limit}&offset={offset}")
        alphas = response.json()["results"]# 是不是要加上region的限制！
        # if alphas[0]["settings"]["region"] == "CHN":
        #     fetched_alphas.extend(alphas)
        # else:
        #     continue
        fetched_alphas.extend(alphas)
            
        #alphaNum += len(alphas)  # Count the total number of fetched alphas
        

        if len(alphas) < limit:
            break
        offset += limit

    return fetched_alphas[:total_alphas]

sim_result = get_n_is_alphas(s,50)

In [35]:
sim_result

[{'id': 'Kklq9ql',
  'type': 'REGULAR',
  'author': 'WD77850',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'USA',
   'universe': 'ILLIQUID_MINVOL1M',
   'delay': 1,
   'decay': 70,
   'neutralization': 'INDUSTRY',
   'truncation': 0.08,
   'pasteurization': 'ON',
   'unitHandling': 'VERIFY',
   'nanHandling': 'ON',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': {'code': "c = implied_volatility_call_150;\r\np = implied_volatility_put_150;\r\ns = c-p;\r\na = group_extra(s*1.5,0.6,industry);\r\ngroup1 = bucket(ts_rank(ts_std_dev(returns,10),252), range = '0,1,0.1');\r\ngroup2 =  bucket(rank(cap), range = '0,1,0.1');\r\ngroup3 =  bucket(rank(assets), range = '0,1,0.1');\r\ncondition1 = if_else(a-group_mean(a,opt4_total_volume,group1)>0,1,0);\r\nd = 3;\r\nv = vector_neut(log(volume/sharesout),cap);\r\n group_rank(ts_decay_exp_window(ts_mean(condition1,d),20,factor = 0.95),group2)\r\n",
   'description': None,
   'operatorCount': 20},
  'dateCreated': '2024-0

取得新alpha 的before or after

拿到旧alpha的Pnl

In [36]:
from time import sleep

def pnl_search(alpha_id):
    finished = False
    while True:
        pnl  = s.get("https://api.worldquantbrain.com/alphas/"+ alpha_id+"/recordsets/pnl")
        if pnl.headers.get("Retry-After", 0) == 0:
            break
        print("Sleeping for " + pnl.headers["Retry-After"] + " seconds")
        sleep(float(pnl.headers["Retry-After"]))
    print("PnL retrieved")
    pnla = pnl.json()
    return pnla


In [37]:
import numpy as np

def corr_count(list1, list2):
    # 移除包含 NoneType 的元素
    valid_indices = [i for i in range(len(list1)) if list1[i] is not None and list2[i] is not None]
    list1_valid = [list1[i] for i in valid_indices]
    list2_valid = [list2[i] for i in valid_indices]
    
    # 使用numpy.corrcoef计算相关性
    correlation_matrix = np.corrcoef(list1_valid, list2_valid)
    
    # 获取相关性矩阵中的相关系数
    correlation_coefficient = correlation_matrix[0, 1]
    
    return correlation_coefficient


In [38]:
def sameLen(list1,list2):
    min_length = min(len(list1), len(list2))
    list1 = list1[-min_length:]
    list2 = list2[-min_length:]
    return list1,list2

In [68]:
list_corr = []
for i in sim_result:
    ida = i["id"]
    pnlA = pnl_search(ida) #这个返回值不知道是不是这样
    pnl_list = []
    records = pnlA["records"]#这是一个大列表 现在要取到所有小列表的索引1
    for lis in records:
        pnl_list.append(lis[1])
    n_pnl_list,n_new = sameLen(pnl_list,before)#这里改before 和after
    list_corr.append(corr_count(n_pnl_list,n_new))#这里改before 和after
    

    

Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
Sleeping for 1.0 seconds
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
Sleeping for 1.0 seconds
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sle

In [69]:
list_corr

[-0.05010830277867126,
 -0.1191408433633155,
 -0.08555367767845959,
 -0.04278390046449597,
 -0.02687469090068781,
 0.010091838950315339,
 0.04300318218120984,
 -0.08528828203225713,
 0.06674403351121831,
 -0.08931243242144477,
 0.02629175033560336,
 0.6497510732401421,
 0.584061910980717,
 0.5965971059769869,
 0.6708825798563182,
 0.6095433786363441,
 0.48276856389826195,
 0.4833246073662789,
 0.5148050352459218,
 0.5947912171597416,
 0.6595277389007791,
 0.6558079110505914,
 0.5632250310302275,
 0.6120998325222918,
 0.5366850950997158,
 0.5176694565355578,
 0.5138664558380867,
 0.5591406566448897,
 0.5558558113590272,
 0.6527844310111521,
 0.44164111264581185,
 0.6820684713330091,
 0.6236612728416224,
 0.7075778518157947,
 0.6347507670699614,
 0.7083755032554767,
 0.6023390874561724,
 0.5448585746612413,
 0.4425717126583612,
 0.7440881752898111,
 0.7633007688588122,
 0.5561808456113715,
 0.30745278792077785,
 0.5645786827985507,
 0.322357703513078,
 0.5856178336635972]

最大相关系数值

In [70]:
# 最大值
max_value = max(list_corr)
print(f"最大值：{max_value}")

# 最小值
min_value = min(list_corr)
print(f"最小值：{min_value}")

# 平均值
average_value = sum(list_corr) / len(list_corr)
print(f"平均值：{average_value}")

最大值：0.7633007688588122
最小值：-0.1191408433633155
平均值：0.4324277765018586


滚动窗口的方法

In [16]:
import pandas as pd

import pandas as pd

def calculate_rolling_correlation(list1, list2, window_size):
    df = pd.DataFrame({'list1': list1, 'list2': list2})
    rolling_correlation_results = df.rolling(window=window_size).corr()
    correlation_main = rolling_correlation_results.loc[:, ('list1', 'list2')]
    return correlation_main


In [23]:
list_corr = []

for i in sim_result:
    ida = i["id"]                                                                                                                                                                                                                                                                                        
    pnlA = pnl_search(ida)
    pnl_list = []

    # 获取 pnl_list，这部分保持不变
    for lis in pnlA["records"]:
        pnl_list.append(lis[1])

    n_pnl_list, n_new = sameLen(pnl_list, before)

    # 计算滚动窗口相关性
    correlation_main = calculate_rolling_correlation(n_pnl_list, n_new, 400)

    specific_column_correlation = correlation_main['list1']
    list_corr.append(specific_column_correlation)

PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved
PnL retrieved


In [24]:
list_corr

[0     list1         NaN
       list2         NaN
 1     list1         NaN
       list2         NaN
 2     list1         NaN
                  ...   
 2406  list2    0.209541
 2407  list1    1.000000
       list2    0.193640
 2408  list1    1.000000
       list2    0.176915
 Name: list1, Length: 4818, dtype: float64,
 0     list1         NaN
       list2         NaN
 1     list1         NaN
       list2         NaN
 2     list1         NaN
                  ...   
 2406  list2    0.202229
 2407  list1    1.000000
       list2    0.190531
 2408  list1    1.000000
       list2    0.178290
 Name: list1, Length: 4818, dtype: float64,
 0     list1         NaN
       list2         NaN
 1     list1         NaN
       list2         NaN
 2     list1         NaN
                  ...   
 2406  list2    0.325240
 2407  list1    1.000000
       list2    0.309708
 2408  list1    1.000000
       list2    0.293913
 Name: list1, Length: 4818, dtype: float64,
 0     list1         NaN
       list2      

方案2

In [39]:
import pandas as pd

def calculate_rolling_correlation(list1, list2, window_size):
    # 创建 DataFrame，同时处理可能存在的空值
    df = pd.DataFrame({'list1': list1, 'list2': list2})
    df = df.dropna()  # 去除包含空值的行
    rolling_correlation_results = df.rolling(window=window_size).corr()
    
    # 选择 'list2' 列的相关性
    correlation_main = rolling_correlation_results.xs('list2', level=1)
    
    return correlation_main

list_corr = []

for i in sim_result:
    ida = i["id"]
    pnlA = pnl_search(ida)
    pnl_list = []

    # 获取 pnl_list，这部分保持不变
    for lis in pnlA["records"]:
        pnl_list.append(lis[1])

    n_pnl_list, n_new = sameLen(pnl_list, before)

    # 计算滚动窗口相关性
    correlation_main = calculate_rolling_correlation(n_pnl_list, n_new, 400)

    # 获取 'list2' 列的最大值并添加到列表
    max_value_list2 = correlation_main.mean().iloc[0]  # 选择第一个值
    list_corr.append(max_value_list2)



Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds
PnL retrieved
Sleeping for 1.0 seconds


In [40]:
list_corr

[0.09664396563781188,
 0.1209167518476167,
 0.11670968764331839,
 0.3118378730176808,
 0.19590961607966395,
 0.3369813517261103,
 0.4128881308965916,
 0.2985720845818595,
 0.32423563067015415,
 0.1428607861555794,
 0.1117838246069347,
 0.11709386464363657,
 0.25650106546766926,
 0.18909406211421617,
 0.123827076198024,
 0.5178819588140761,
 0.10343833467034287,
 0.06508931668254275,
 0.1714380710308014,
 0.37192349471810326,
 0.39343423866889704,
 0.13772514009170939,
 0.11109725000046751,
 0.3952166403782023,
 0.3542029607755554,
 0.28665296903107707,
 0.05819935756614167,
 0.21623087775138067,
 -0.16430157393891057,
 0.22038208253430416,
 0.16513661868229146,
 0.28642694444414446,
 0.06479791017397071,
 0.48296196452427764,
 0.1290546737325264,
 0.29348854668107727,
 0.3563467458889437,
 0.10160202229308161,
 0.13824269730733266,
 0.6654833084807836,
 0.6978856889022109,
 0.2333042995351409,
 0.18697628416691353,
 0.5392032447062828,
 0.24701665233008005,
 0.2933682857720896]

In [41]:
# 最大值
max_value = max(list_corr)
print(f"最大值：{max_value}")

# 最小值
min_value = min(list_corr)
print(f"最小值：{min_value}")

# 平均值
average_value = sum(list_corr) / len(list_corr)
print(f"平均值：{average_value}")

最大值：0.6978856889022109
最小值：-0.16430157393891057
平均值：0.24512527777571097


In [42]:
final_score_b =calculate_score(before_after_yearlyStats['before'],returns1,turnover1)

In [43]:
final_score_a = calculate_score(before_after_yearlyStats['after'],returns2,turnover2)*max_value

待完善

In [ ]:
score_rank =  calculate_score(before_after_yearlyStats['after'],returns2,turnover2)*max_value

In [ ]:
score_sign  =  calculate_score(before_after_yearlyStats['after'],returns2,turnover2)*max_value

In [45]:
def update_alpha_color(session, alpha_id,color,final_score_a,final_score_b):
    update_data = {"color":color}
    response = session.patch(f"https://api.worldquantbrain.com/alphas/{alpha_id}", json=update_data)
    return response.status_code == 200
if final_score_a>final_score_b*0.8:
    update_alpha_color(session = s, alpha_id = alpha_id, color = 'GREEN',final_score_a = final_score_a,final_score_b = final_score_b)
else:
    print("The alpha is not avery good")
    update_alpha_color(session = s, alpha_id = alpha_id, color = 'RED',final_score_a = final_score_a,final_score_b = final_score_b)
    

The alpha is not avery good


New SCORE

In [ ]:
def update_alpha_color(session, alpha_id,color,final_score_a,final_score_b,score_rank,socre_sign):
    update_data = {"color":color}
    response = session.patch(f"https://api.worldquantbrain.com/alphas/{alpha_id}", json=update_data)
    return response.status_code == 200
if (final_score_a*0.6+score_rank*0.2+score_sign*0.2)>final_score_b*0.8:
    update_alpha_color(session = s, alpha_id = alpha_id, color = 'GREEN',final_score_a = final_score_a,final_score_b = final_score_b)
else:
    print("The alpha is not avery good")
    update_alpha_color(session = s, alpha_id = alpha_id, color = 'RED',final_score_a = final_score_a,final_score_b = final_score_b)
    